# **EC9640 - Artificial Intelligence Project**
### **Project Name** - Spelling corrector and grammar checker for Tamil
### **Team Member** - 2020/E/067 & 2020/E/145

# **Rule Based  - Grammar Checker Implementation**

## Import nessary library

In [9]:
# Install required libraries
!pip install stanza

import stanza

# Download Tamil model for Stanza
stanza.download('ta')

# Initialize the Stanza pipeline for Tamil
nlp = stanza.Pipeline(lang='ta', processors='tokenize,pos,lemma,depparse')


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ta (Tamil) ...
INFO:stanza:File exists: /root/stanza_resources/ta/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ta (Tamil):
| Processor | Package      |
----------------------------
| tokenize  | ttb          |
| mwt       | ttb          |
| pos       | ttb_nocharlm |
| lemma     | ttb_nocharlm |
| depparse  | ttb_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no l

## Prepare Verbs Dictionary

In [152]:
tamil_dictionary_verbs = [
    # Past Tense
    "படித்தேன்", "படித்தோம்", "படித்தாய்", "படித்தீர்கள்", "படித்தான்", "படித்தாள்",
    "படித்தார்கள்", "படித்தது", "படித்தன",

    # Present Tense
    "படிக்கிறேன்", "படிக்கிறோம்", "படிக்கிறாய்", "படிக்கிறீர்கள்", "படிக்கிறான்",
    "படிக்கிறாள்", "படிக்கிறார்கள்", "படிக்கிறது", "படிக்கின்றன",

    # Future Tense
    "படிப்பேன்", "படிப்போம்", "படிப்பாய்", "படிப்பீர்கள்", "படிப்பான்", "படிப்பாள்",
    "படிப்பார்கள்", "படிக்கும்",

    # # Imperative Mood
    # "படி", "படிங்க",

    # # Negative Verbs
    # "படிக்கவில்லை", "படிக்கமாட்டேன்",

    # # Gerund and Other Forms
    # "படித்து", "படிக்க", "படித்தல்"

    # Past Tense
    "சென்றேன்", "சென்றோம்", "சென்றாய்", "சென்றீர்கள்", "சென்றான்", "சென்றாள்",
    "சென்றார்கள்", "சென்றது", "சென்றன",

    # Present Tense
    "செல்கிறேன்", "செல்கிறோம்", "செல்கிறாய்", "செல்கிறீர்கள்", "செல்கிறான்",
    "செல்கிறாள்", "செல்கிறார்கள்", "செல்கிறது", "செல்கின்றன",

    # Future Tense
    "செல்வேன்", "செல்வோம்", "செல்வாய்", "செல்வீர்கள்", "செல்வான்", "செல்வாள்",
    "செல்வார்கள்", "செல்லும்",

    # # Imperative Mood
    # "செல்", "செல்லுங்கள்",

    # # Negative Verbs
    # "செல்லவில்லை", "செல்லமாட்டேன்",

    # # Gerund and Other Forms
    # "சென்று", "செல்ல", "சென்றல்"

    # Past Tense
    "செய்தேன்", "செய்தோம்", "செய்தாய்", "செய்தீர்கள்", "செய்தான்", "செய்தாள்","செய்தார்",
    "செய்தார்கள்", "செய்தது", "செய்தன",

    # Present Tense
    "செய்கிறேன்", "செய்கிறோம்", "செய்கிறாய்", "செய்கிறீர்கள்", "செய்கிறான்", "செய்கிறார்",
    "செய்கிறாள்", "செய்கிறார்கள்", "செய்கிறது", "செய்கின்றன",

    # Future Tense
    "செய்வேன்", "செய்வோம்", "செய்வாய்", "செய்வீர்கள்", "செய்வான்", "செய்வாள்", "செய்வார்",
    "செய்வார்கள்", "செய்யும்",

    # Imperative Mood
    # "செய்", "செய்யுங்கள்",

    # # Negative Verbs
    # "செய்யவில்லை", "செய்யமாட்டேன்",

    # # Gerund and Other Forms
    # "செய்து", "செய்ய", "செய்தல்"


    # Past Tense
    "இருந்தேன்", "இருந்தோம்", "இருந்தாய்", "இருந்தீர்கள்", "இருந்தான்", "இருந்தாள்", "இருந்தார்",
    "இருந்தார்கள்", "இருந்தது", "இருந்தன",

    # Present Tense
    "இருக்கிறேன்", "இருக்கிறோம்", "இருக்கிறாய்", "இருக்கிறீர்கள்", "இருக்கிறான்", "இருக்கிறாள்",
    "இருக்கிறார்", "இருக்கிறார்கள்", "இருக்கிறது", "இருக்கின்றன",

    # Future Tense
    "இருப்பேன்", "இருப்போம்", "இருப்பாய்", "இருப்பீர்கள்", "இருப்பான்", "இருப்பாள்",
    "இருப்பார்", "இருப்பார்கள்", "இருக்கும்",

    # Imperative Mood
    # "இரு", "இருங்கள்",

    # Negative Verbs
    # "இருக்கவில்லை", "இருக்கமாட்டேன்",

    # Gerund and Other Forms
    # "இருந்து", "இருக்க", "இருத்தல்"
]

print(tamil_dictionary_verbs)

['படித்தேன்', 'படித்தோம்', 'படித்தாய்', 'படித்தீர்கள்', 'படித்தான்', 'படித்தாள்', 'படித்தார்கள்', 'படித்தது', 'படித்தன', 'படிக்கிறேன்', 'படிக்கிறோம்', 'படிக்கிறாய்', 'படிக்கிறீர்கள்', 'படிக்கிறான்', 'படிக்கிறாள்', 'படிக்கிறார்கள்', 'படிக்கிறது', 'படிக்கின்றன', 'படிப்பேன்', 'படிப்போம்', 'படிப்பாய்', 'படிப்பீர்கள்', 'படிப்பான்', 'படிப்பாள்', 'படிப்பார்கள்', 'படிக்கும்', 'சென்றேன்', 'சென்றோம்', 'சென்றாய்', 'சென்றீர்கள்', 'சென்றான்', 'சென்றாள்', 'சென்றார்கள்', 'சென்றது', 'சென்றன', 'செல்கிறேன்', 'செல்கிறோம்', 'செல்கிறாய்', 'செல்கிறீர்கள்', 'செல்கிறான்', 'செல்கிறாள்', 'செல்கிறார்கள்', 'செல்கிறது', 'செல்கின்றன', 'செல்வேன்', 'செல்வோம்', 'செல்வாய்', 'செல்வீர்கள்', 'செல்வான்', 'செல்வாள்', 'செல்வார்கள்', 'செல்லும்', 'செய்தேன்', 'செய்தோம்', 'செய்தாய்', 'செய்தீர்கள்', 'செய்தான்', 'செய்தாள்', 'செய்தார்', 'செய்தார்கள்', 'செய்தது', 'செய்தன', 'செய்கிறேன்', 'செய்கிறோம்', 'செய்கிறாய்', 'செய்கிறீர்கள்', 'செய்கிறான்', 'செய்கிறார்', 'செய்கிறாள்', 'செய்கிறார்கள்', 'செய்கிறது', 'செய்கின்றன', 'செய்வேன்', 'செய்

## Tense Related words to find the actual tense

In [79]:

tense_related_words = {
  "past": [
      "நேற்று",
      "நேற்றைய",
      "முந்தினம்",
      "சில நாட்களுக்கு முன்",
      "சில நாட்களுக்கு முன்பு",
      "சில நேரத்திற்கு முன்",
      "சில நேரத்திற்கு முன்பு",
      "சில நேரங்களுக்கு முன்",
      "சில நேரங்களுக்கு முன்பு"
    ],
  "present": [
      "இன்று",
      "இப்பொழுது",
      "இவ்வேளை",
      "இவ்வேளைக்கு",
      "இவ்வேளையில்",
      "இன்றைக்கு",
      "தற்போது",
      "தற்போதைய"
  ],
  "future":  [
      "நாளை",
      "நாளைக்கு",
      "எதிர்காலம்",
      "எதிர்வரும்",
      "எதிர்காலத்தில்",
      "பின்னர்",
      "ஒருநாள்",
      "பிறகு",
      "அடுத்த"
    ]
}


## Possible tense ending words for all pronounns

In [81]:
tense_ending_words = {
    'past':{
        'நான்'      : ['தேன்','றேன்','னேன்','டேன்'], # 'றேன்'
        'நாம்'      : ['தோம்','றோம்','னோம்','டோம்'], # 'றோம்'
        'நாங்கள்'  : ['தோம்','றோம்','னோம்','டோம்'], # 'றோம்'
        'நீ'         : ['தாய்','றாய்','னாய்','டாய்'],
        'நீங்கள்'    : ['தீர்கள்','றீர்கள்','னீர்கள்','டீர்கள்'], # 'றீர்கள்'
        'அவன்'     : ['தான்','றான்','னான்','டான்'], #'றான்'
        'அவள்'     : ['தாள்','றாள்','னாள்','டாள்'], # 'றாள்'
        'அவர்கள்'  : ['தார்கள்','றார்கள்','னார்கள்','டனர்','டார்கள்'], #'றார்கள்'
        'அவர்'      : ['தார்','றார்','னார்','டார்'],  # 'றார்'
        'அது'      : ['தது','றது','யது','டது'], #'றது'
        'அவை'    :['தன','றன','டின','டன'], #'றன'
        'அவைகள்' :['தன','றன','டின','டன'] #'றன'
    },
    'present':{
        'நான்': ['கிறேன்','கின்றேன்'], #'கிறேன்','கின்றேன்'
        'நாம்': ['கிறோம்','கின்றோம்'], #'கிறோம்','கின்றோம்'
        'நாங்கள்': ['கிறோம்','கின்றோம்'], #'கிறோம்','கின்றோம்'
        'நீ':  ['கிறாய்','கின்றாய்'],
        'நீங்கள்': ['கிறீர்கள்','கின்றீர்கள்'],
        'அவன்': ['கிறான்','கின்றான்'],
        'அவள்': ['கிறாள்','கின்றாள்'],
        'அவர்கள்': ['கிறார்கள்','கின்றார்கள்'],
        'அவர்': ['கிறார்','கின்றார்'],
        'அது': ['கிறது','கின்றது'],
        'அவை':['கிறன','கின்றன'],
        'அவைகள்':['கிறன','கின்றன']
    },
    'future':{
        'நான்': ['பேன்','வேன்'],
        'நாம்': ['போம்','வோம்'],
        'நாங்கள்': ['போம்','வோம்'],
        'நீ': ['பாய்','வாய்'],
        'நீங்கள்': ['பீர்கள்','வீர்கள்'],
        'அவன்': ['பான்','வான்'],
        'அவள்': ['பாள்','வாள்'],
        'அவர்கள்': ['பார்கள்','வார்கள்'],
        'அவர்': ['பார்','வார்'],
        'அது': ['கும்','லும்'],
        'அவை':['கும்','லும்'],
        'அவைகள்':['கும்','லும்']
    },

}

## List of all ending words for a verb

In [82]:
# Extract lists for each tense
past_list = [item for sublist in tense_ending_words['past'].values() for item in sublist]
present_list = [item for sublist in tense_ending_words['present'].values() for item in sublist]
future_list = [item for sublist in tense_ending_words['future'].values() for item in sublist]
all_verb_endings_words = present_list + past_list + future_list

print("Past:", past_list)
print("Present:", present_list)
print("Future:", future_list)
print("All ending words:", all_verb_endings_words)

Past: ['தேன்', 'றேன்', 'னேன்', 'டேன்', 'தோம்', 'றோம்', 'னோம்', 'டோம்', 'தோம்', 'றோம்', 'னோம்', 'டோம்', 'தாய்', 'றாய்', 'னாய்', 'டாய்', 'தீர்கள்', 'றீர்கள்', 'னீர்கள்', 'டீர்கள்', 'தான்', 'றான்', 'னான்', 'டான்', 'தாள்', 'றாள்', 'னாள்', 'டாள்', 'தார்கள்', 'றார்கள்', 'னார்கள்', 'டனர்', 'டார்கள்', 'தார்', 'றார்', 'னார்', 'டார்', 'தது', 'றது', 'யது', 'டது', 'தன', 'றன', 'டின', 'டன', 'தன', 'றன', 'டின', 'டன']
Present: ['கிறேன்', 'கின்றேன்', 'கிறோம்', 'கின்றோம்', 'கிறோம்', 'கின்றோம்', 'கிறாய்', 'கின்றாய்', 'கிறீர்கள்', 'கின்றீர்கள்', 'கிறான்', 'கின்றான்', 'கிறாள்', 'கின்றாள்', 'கிறார்கள்', 'கின்றார்கள்', 'கிறார்', 'கின்றார்', 'கிறது', 'கின்றது', 'கிறன', 'கின்றன', 'கிறன', 'கின்றன']
Future: ['பேன்', 'வேன்', 'போம்', 'வோம்', 'போம்', 'வோம்', 'பாய்', 'வாய்', 'பீர்கள்', 'வீர்கள்', 'பான்', 'வான்', 'பாள்', 'வாள்', 'பார்கள்', 'வார்கள்', 'பார்', 'வார்', 'கும்', 'லும்', 'கும்', 'லும்', 'கும்', 'லும்']
All ending words: ['கிறேன்', 'கின்றேன்', 'கிறோம்', 'கின்றோம்', 'கிறோம்', 'கின்றோம்', 'கிறாய்', 'கின்றாய்',

## Download the embedding model

In [1]:
import os
if not os.path.exists("/content/cc.ta.300.bin"):
  !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.bin.gz
  !gunzip cc.ta.300.bin.gz

In [1]:
!pip install fasttext
import fasttext
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained binary FastText embeddings
embedding_path = "cc.ta.300.bin"  # Path to pre-trained Tamil FastText embeddings
model = fasttext.load_model(embedding_path)

## Function for find the pronoun for subject

In [90]:
def find_subject_pronoun(subject):
  person = None
  number = None
  gender = None  # Add gender variable
  subject_features = subject.feats or ""

  # find the person
  if "Person=1" in subject_features:
      person = 1
  elif "Person=2" in subject_features:
      person = 2
  elif "Person=3" in subject_features:
    person = 3

  # find the numbers
  if "Number=Sing" in subject_features:
      number = "singular"
  elif "Number=Plur" in subject_features:
      number = "plural"

  # find the gender (if available)
  if "Gender=Masc" in subject_features:
      gender = "masculine"
  elif "Gender=Fem" in subject_features:
      gender = "feminine"
  elif "Gender=Neut" in subject_features:  # Add neuter gender
      gender = "neuter"
  elif  "Gender=Com" in subject_features:  # Add common gender
      gender = "common"

  ########################################################
  # Print details
  # print('Person : ',person)
  # print('Number : ',number)
  # print('Gender : ',gender)
  ########################################################

  # Pronoun mapping based on person, number, and gender
  pronoun_mapping = {
      (1, "singular"): "நான்",
      (1, "plural"): "நாங்கள்",
      (2, "singular"): "நீ",
      (2, "plural"): "நீங்கள்",
      (3, "singular", "masculine"): "அவன்",
      (3, "singular", "feminine"): "அவள்",
      (3, "singular", "common"): "அவர்",
      (3, "singular", "neuter"): "அது",  # Add neuter singular
      (3, "plural"): "அவர்கள்",
      (3, "plural", "neuter"): "அவைகள்" # Add neuter plural

  }

  pronoun = pronoun_mapping.get((person, number, gender), pronoun_mapping.get((person, number)))
  # Try to get pronoun with gender, if not found, try without gender

  return [pronoun, person,number,gender]

## Function for finding the tense of the verb

In [88]:
import random

def find_verb_tense(verb):
  tense = None
  tense_pos = None
  tense_ends = None

  # find the tense using verb features
  verb_features = verb.feats or ""
  if "Tense" in verb_features:
    if "Tense=Pres" in verb_features:
      tense_pos = "present"
    elif "Tense=Past" in verb_features:
        tense_pos = "past"
    elif "Tense=Fut" in verb_features:
        tense_pos = "future"

  # find tense using ending words
  if any(verb.text.endswith(suffix) for suffix in present_list):
      tense_ends = "present"
  elif any(verb.text.endswith(suffix) for suffix in past_list):
      tense_ends = "past"
  elif any(verb.text.endswith(suffix) for suffix in future_list):
      tense_ends = "future"



  # Finalize the tense
  if tense_pos and tense_ends:
    if tense_pos == tense_ends:
      tense = tense_pos
    else:
      tense = tense_ends
  elif tense_pos:
    tense = tense_pos
  elif tense_ends:
    tense = tense_ends
  else:
    print('tense not found.............')
    # tense = random.choice(["past", "present", "future"])
    tense = "Unknown"
  #############################
  # print('tense_pos : ',tense_pos)
  # print('tense_ends : ',tense_ends)
  # print('verb_tense : ',tense)
  #############################

  return tense

## Function for generating possible verb using suffix

In [85]:
# Function to find the consonant from a letter
def get_consonant(letter):
    # Check if the letter ends with a pulli (்), return the consonant part
    if letter[-1] == '்':  # Pulli mark
        return letter
    return letter  # If no pulli, return as is

# Function to correct the word and generate all possible corrected verbs
def generate_possible_verbs(verb, wrong_suffix, possible_suffixes,actual_tense,verb_tense):
    possible_verbs = []
    word = verb.text
    lemma = verb.lemma
    diff_base_verb = None
    current_base_verb = None
    if lemma in base_verbs.keys():
      diff_base_verb = base_verbs[lemma][actual_tense]
      current_base_verb = base_verbs[lemma][verb_tense]
      # print('diff_base_verb : ',diff_base_verb)
      # print('current_base_verb : ',current_base_verb)

    if diff_base_verb and diff_base_verb != lemma:
      lemmas = [diff_base_verb, lemma]
    else:
      lemmas = [lemma]

    # print('lemmas : ',lemmas)

    for verb_lemma in lemmas:
       if word.endswith(wrong_suffix):
          # Remove the incorrect suffix to get the base word
          base_word = word[:-len(wrong_suffix)]

          # Extract the last consonant from the base word
          last_consonant_base = get_consonant(base_word[-1])

          # print('verb_lemma : ', verb_lemma)
          # print('base_word : ', base_word)
          # print('last_consonant_base : ', last_consonant_base)

          # check whether base_word and verb_lemma are not same
          if base_word != verb_lemma and len(base_word) != len(verb_lemma) :
            # Check if last consonant is ்
            if last_consonant_base == '்':
                # Remove the last two letters from the base word
                base_word = base_word[:-1]  # Remove pulli mark
                base_word = base_word[:-1]  # Remove the last consonant

                # print('Updated base_word after removing last two letters: ', base_word)

                # Generate all possible corrected verbs with pulli
                for suffix in possible_suffixes:
                    first_consonant_suffix = get_consonant(suffix[0])
                    corrected_verb = base_word + first_consonant_suffix + '்' + suffix
                    possible_verbs.append(corrected_verb)
                # Generate all possible corrected verbs without pulli
                for suffix in possible_suffixes:
                    first_consonant_suffix = get_consonant(suffix[0])
                    corrected_verb = base_word + suffix
                    possible_verbs.append(corrected_verb)

          else:
            if current_base_verb:
              base_word = base_word.replace(current_base_verb,verb_lemma )
            else:
              base_word = base_word.replace(verb_lemma,verb_lemma )
            # Generate all possible corrected verbs with pulli
            for suffix in possible_suffixes:
                first_consonant_suffix = get_consonant(suffix[0])
                corrected_verb = base_word + first_consonant_suffix + '்' + suffix
                possible_verbs.append(corrected_verb)
            # Generate all possible corrected verbs without pulli
            for suffix in possible_suffixes:
                first_consonant_suffix = get_consonant(suffix[0])
                corrected_verb = base_word + suffix
                possible_verbs.append(corrected_verb)


    return possible_verbs



## Base verb vary with tenses

In [84]:
base_verbs = {
    'செல்': {
        'past':'சென்',
        'present':'செல்',
        'future': 'சென்'
    },
    'ஓடு': {
        'past':'ஓடி',
        'present':'ஓடு',
        'future': 'ஓடு'
    }
}

## Function for find the most suitable verb from possible verbs

### Method 1 : words in model

In [77]:
import fasttext
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Function to find the most suitable word
def find_most_suitable_word_for_all_1(inputs, dictionary, model):
    ##################################
    # print('inputs : ',inputs)
    # print('dictionary : ',dictionary)
    # print('model : ',model)
    ##################################
    # Calculate the average embedding for all input words
    input_vectors = []
    for word in inputs:
        # print('word in input : ',word)
        if word in model:  # Skip words not in the model
            # print('word in model : ',word)
            input_vectors.append(model.get_word_vector(word))
    if not input_vectors:
        return None  # No valid embeddings for inputs
    average_input_vector = np.mean(input_vectors, axis=0)

    # Find the dictionary word with the highest similarity to the average embedding
    best_match = None
    highest_similarity = -1

    # print('dictionary : ',dictionary)

    for dict_word in dictionary:
        # print("dict_word: ",dict_word)
        dict_vector = model.get_word_vector(dict_word)
        similarity = cosine_similarity([average_input_vector], [dict_vector])[0][0]
        # print('similarity : ',similarity)
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = dict_word

    return best_match

# verbs = ['செய்த்தார்', 'செய்ற்றார்', 'செய்ன்னார்', 'செய்ட்டார்']
# test = find_most_suitable_word_for_all(verbs, tamil_dictionary_verbs, model)
# print('correct verb : ',test)

### Method 2 : words not in the model

In [78]:
import fasttext
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to find the most suitable word from the dictionary
def find_most_suitable_word_for_all_2(inputs, dictionary, model):
    # Generate embeddings for inputs
    input_vectors = []
    for word in inputs:
        # print(f"Generating embedding for: {word}")
        input_vectors.append(model.get_word_vector(word))  # Generate embedding even for OOV words

    # Compute the average embedding of input words
    average_input_vector = np.mean(input_vectors, axis=0)

    # Precompute embeddings for the dictionary
    dictionary_embeddings = {}
    for dict_word in dictionary:
        # print(f"Generating embedding for dictionary word: {dict_word}")
        dictionary_embeddings[dict_word] = model.get_word_vector(dict_word)

    # Find the dictionary word with the highest similarity
    best_match = None
    highest_similarity = -1
    for dict_word, dict_vector in dictionary_embeddings.items():
        similarity = cosine_similarity([average_input_vector], [dict_vector])[0][0]
        # print(f"Similarity between input and '{dict_word}': {similarity}")
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = dict_word

    return best_match

# # Example inputs and dictionary
# verbs = ['செய்த்தார்', 'செய்ற்றார்', 'செய்ன்னார்', 'செய்ட்டார்']  # Generated verbs
# # tamil_dictionary_verbs = ["செய்தான்", "செய்கிறான்", "செய்வான்", "செய்யும்"]  # Prepared dictionary verbs


# # Find the most suitable verb
# correct_verb = find_most_suitable_word_for_all(verbs, tamil_dictionary_verbs, model)
# print("Correct Verb:", correct_verb)


## Grammar checker Implementation

In [137]:
def grammar_checker(sentence_data):
    """
    Check and correct grammar errors.
    """
    corrections = []
    subject_pronoun = []
    subject_noun = []
    subject = None
    verb = None
    adverb = None
    actual_tense = None
    verb_tense = None
    number = None
    gender = None
    pronoun = None


    # Identify subject, verb and adverb
    for word in sentence_data.words:
        if word.deprel == "nsubj" and word.upos == "PRON":  # Prioritize pronoun as the subject
            subject_pronoun.append(word)
        elif word.deprel == "nsubj" and word.upos == "NOUN":
            subject_noun.append(word)
        elif word.deprel == "root" and word.upos == "VERB":  # Root verb
            verb = word
        if word.upos == "ADV":  # Adverb
            adverb = word

    # handle if both noun and pronoun are found as subject
    if subject_pronoun and subject_noun:
        subject = subject_pronoun[0]
    elif subject_pronoun:
        subject = subject_pronoun[0]
    elif subject_noun:
        subject = subject_noun[0]
    else:
        subject = [word for word in sentence_data.words if word.upos in ("NOUN", "PRON","PROPN")]
        subject=subject[0]

    #################################
    if subject:
      # print('subject: ',subject.text)
      pronoun, person,number,gender = find_subject_pronoun(subject)
      # print('pronoun : ',pronoun)
    # if verb: print('verb: ',verb.text)
    # print('verb: ',verb.text) if verb else print('No verb')
    # print('adverb: ',adverb.text) if adverb else print('No adverb')
    #################################



    # find the verb tense
    if verb:
      verb_tense = find_verb_tense(verb)


    # find the actual tense
    if adverb:
      if adverb and adverb.lemma in tense_related_words["past"]:
        actual_tense = "past"
      elif adverb and adverb.lemma in tense_related_words["present"]:
        actual_tense = "present"
      elif adverb and adverb.lemma in tense_related_words["future"]:
        actual_tense = "future"
      else:
        actual_tense = verb_tense
    else:
        actual_tense = verb_tense

    # print('actual_tense : ',actual_tense)

    # Check the tense error and display it
    is_tense_error = False
    if(actual_tense != verb_tense):
      is_tense_error = True
      print(f'\nTENSE ERROR :\nThe sentence should be in {actual_tense} but the verb tense is {verb_tense}')

    # check Subject-Verb Agreement error and display it
    is_subject_Verb_Agreement_error = None
    if subject and verb:
      endind_words = tense_ending_words[verb_tense][pronoun]
      ##############################################
      # print(f'\nchecking ending words...............')
      # print(f'Ending words for {pronoun} are: {endind_words}')
      ##############################################
      is_subject_Verb_Agreement_error = True
      for ending_word in endind_words:
        # print('ending word : ',ending_word)
        if verb.text.endswith(ending_word):
          # print(f'*************ENDING WORD MATCH: {ending_word}************')
          is_subject_Verb_Agreement_error = False
          break
      if is_subject_Verb_Agreement_error:
        print(f'\nSUBJECT VERB AGREEMENT ERROR:\n{subject.text} should not be ended with {verb.text}')


    # Correct the error

    # find the wrong_suffix
    if is_tense_error or is_subject_Verb_Agreement_error:
      wrong_suffix = None
      for ending_word in all_verb_endings_words:
        if verb.text.endswith(ending_word):
          wrong_suffix = ending_word
          break
      ##################################
      # print('\nwrong_suffix: ',wrong_suffix)
      ##################################

      verb_combinations = []
      possible_suffixes = tense_ending_words[actual_tense][pronoun]


      # if both error occur
      # if is_tense_error and is_subject_Verb_Agreement_error or is_subject_Verb_Agreement_error:

      # if there any error
      if is_tense_error or is_subject_Verb_Agreement_error:
        verb_combinations = generate_possible_verbs(verb, wrong_suffix, possible_suffixes,actual_tense,verb_tense)
        # Find the most suitable word from all inputs
        correct_verb = None
        correct_verb = find_most_suitable_word_for_all_1(verb_combinations, tamil_dictionary_verbs, model)
        if correct_verb is None:
          correct_verb = find_most_suitable_word_for_all_2(verb_combinations, tamil_dictionary_verbs, model)
        ###############################################
        # print('possible_suffixes: ',possible_suffixes)
        # print('verb_combinations: ',verb_combinations)
        # print('correct_verb: ',correct_verb)
        ###############################################
        # Correctly join tokens without spaces before punctuation
        original_sentence = "".join([
            token.text if token.text in ".,!?;:\"" else f" {token.text}"
            for token in sentence_data.tokens
        ]).strip()

        corrcted_sentence = original_sentence.replace(verb.text, correct_verb)

        # print(f'\nOriginal sentence: {original_sentence}')
        # print(f'Grammar suggestion: {corrcted_sentence}')

        return corrcted_sentence

      # if subject verb aggreement error occur
      # elif is_subject_Verb_Agreement_error:
    else:
      original_sentence = "".join([
            token.text+" " if token.text in ".,!?;:\"" else f" {token.text}"
            for token in sentence_data.tokens
        ]).strip()
      print(f'\nNo error in the sentence: {original_sentence}')

      return original_sentence




In [140]:
# Enhanced Grammar Checker
def check_grammar(original_para):
    """
    Check and correct grammatical errors in a Tamil sentence.
    """
    print(f"\nOriginal para: \n{original_para}")

    doc = nlp(original_para)
    corrected_para = ""
    # sentence_count = 1

    for sentence_data in doc.sentences:
        # print(f"\nSentence {sentence_count}:")
        # sentence_count += 1
        corrected_para = corrected_para + grammar_checker(sentence_data)


    print(f"\ncorrected para: \n{corrected_para}")

    # return corrected_para



## Test the Grammer checker

### Define test paras

In [146]:
para1 = "நான் இன்று காலை பள்ளிக்குச் சென்றேன். நான் நேற்று மாலை சந்தைக்கு சென்றோம்."
para2 = "நீங்கள் நேற்று வேலை செய்தான். நான் நேற்று என் புத்தகம் படிக்கிறேன்."
para3 = "அவள் காலை உணவுகளை சமைக்கவில்லை. அவர் தன் வேலை செய்வான்."
para4 = "நாம் பாட்டு பாடுவார்கள். அவை மிகுந்த மகிழ்ச்சியாக இருந்தது."
para5 = "அவர்கள் இன்று படித்தான். நான் நேற்று காலையில் படிப்போம்."


### Test Para 01

In [147]:
check_grammar(para1)


Original para: 
நான் இன்று காலை பள்ளிக்குச் சென்றேன். நான் நேற்று மாலை சந்தைக்கு சென்றோம்.

TENSE ERROR :
The sentence should be in present but the verb tense is past

SUBJECT VERB AGREEMENT ERROR:
நான் should not be ended with சென்றோம்

corrected para: 
நான் இன்று காலை பள்ளிக்குச் செல்கிறேன்.நான் நேற்று மாலை சந்தைக்கு சென்றேன்.


### Test Para 02

In [148]:
check_grammar(para2)


Original para: 
நீங்கள் நேற்று வேலை செய்தான். நான் நேற்று என் புத்தகம் படிக்கிறேன்.

SUBJECT VERB AGREEMENT ERROR:
நீங்கள் should not be ended with செய்தான்

TENSE ERROR :
The sentence should be in past but the verb tense is present

corrected para: 
நீங்கள் நேற்று வேலை செய்தீர்கள்.நான் நேற்று என் புத்தகம் படித்தேன்.


### Test Para 03

In [149]:
check_grammar(para3)


Original para: 
அவள் காலை உணவுகளை சமைக்கவில்லை. அவர் தன் வேலை செய்வான்.

No error in the sentence: அவள் காலை உணவுகளை சமைக்கவில்லை.

SUBJECT VERB AGREEMENT ERROR:
அவர் should not be ended with செய்வான்

corrected para: 
அவள் காலை உணவுகளை சமைக்கவில்லை.அவர் தன் வேலை செய்வார்.


### Test Para 04

In [150]:
check_grammar(para4)


Original para: 
நாம் பாட்டு பாடுவார்கள். அவை மிகுந்த மகிழ்ச்சியாக இருந்தது.

SUBJECT VERB AGREEMENT ERROR:
நாம் should not be ended with பாடுவார்கள்

SUBJECT VERB AGREEMENT ERROR:
அவை should not be ended with இருந்தது

corrected para: 
நாம் பாட்டு படிப்போம்.அவை மிகுந்த மகிழ்ச்சியாக இருந்தன.


### Test Para 05

In [151]:
check_grammar(para5)


Original para: 
அவர்கள் இன்று படித்தான். நான் நேற்று காலையில் படிப்போம்.

TENSE ERROR :
The sentence should be in present but the verb tense is past

SUBJECT VERB AGREEMENT ERROR:
அவர்கள் should not be ended with படித்தான்

TENSE ERROR :
The sentence should be in past but the verb tense is future

SUBJECT VERB AGREEMENT ERROR:
நான் should not be ended with படிப்போம்

corrected para: 
அவர்கள் இன்று படிக்கிறார்கள்.நான் நேற்று காலையில் படித்தேன்.
